#### read atom positions from .ion file

In [ ]:
def parse_ion_file(file):
    positions = []
    with open(file, 'r') as f:
        for line in f:
            if not 'BLOCK POSITIONS' in line:
                pass
            else:
                positions = parse_position_section(f)
                break
    return(positions)
                
def parse_position_section(fs):
    pos = []
    for line in fs:
        if not 'END BLOCK POSITIONS' in line:
            coords = line.split()[1:4]
            coords = [float(c) for c in coords]
            pos.append(coords)
        else:
            break
    return(pos)

In [ ]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/ion_step0.dat'
pos = parse_ion_file(file)
pos

In [ ]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/md_nve.ion'
parse_ion_file(file)

#### read velocities from PROFESS

In [ ]:
def parse_velocity_file(file):
    velocities = None
    with open(file, 'r') as f:
        for line in f:
            if not 'ION VELOCITIES (a.u.)' in line:
                pass
            else:
                velocities = parse_velocity_section(f)
                break
    return(velocities)
                
def parse_velocity_section(fs):
    velocities = []
    for line in fs:
        velocities_atom = line.split()
        velocities_atom = [float(v) for v in velocities_atom]
        velocities.append(velocities_atom)
    return(velocities)

In [ ]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/vel_step0.dat'
vel = parse_velocity_file(file)
vel

#### read forces from PROFESS

In [ ]:
def parse_force_file(file):
    forces = []
    with open(file, 'r') as f:
        for line in f:
            if not 'TOTAL-FORCE' in line:
                pass
            else:
                f.readline()
                f.readline()
                forces = parse_force_section(f)
                break
    return(forces)

def parse_force_section(f):
    forces = []
    for line in f:
        if not '........' in line:
            forces_atom = line.split()[2:5]
            forces_atom = [float(for_a) for for_a in forces_atom]
            forces.append(forces_atom)
        else:
            break
    return(forces)

In [ ]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/force_step0.dat'
parse_force_file(file)

#### generate profess .ion from ase output

In [3]:
import numpy as np

In [10]:
def generate_cell_section(cell_par):
    cell_section = []
    cell_section.append('%BLOCK LATTICE_CART\n')
    for cell_vec in cell_par:
        cell_vec_string = '\t' + '\t'.join(map(str, cell_vec))
        cell_section.append(cell_vec_string+'\n')
    cell_section.append('%END BLOCK LATTICE_CART\n')
    return(cell_section)

def generate_position_section(atom_types, positions, pos_type):
    positions_section = []
    positions_section.append(f'%BLOCK POSITIONS_{pos_type}\n')
    
    for at, pos in zip(atom_types, positions):
        atom_string = f'\t{at}\t' + '\t'.join(map(str, pos))
        positions_section.append(atom_string+'\n')
    positions_section.append(f'%END BLOCK POSITIONS_{pos_type}\n')
    return(positions_section)

def generate_pp_section(elements, pp_names):
    pp_section = []
    pp_section.append('%BLOCK SPECIES_POT\n')
    for el, pp in zip(elements, pp_names):
        pp_section.append(f'\t{el}\t{pp}\n')
    pp_section.append('%END BLOCK SPECIES_POT\n')
    return(pp_section)
    
def generate_ion_file(cell_par, atom_types, positions, pos_type, pp):
    cell_section = generate_cell_section(cell_par)
    position_section = generate_position_section(atom_types, positions, pos_type)
    elements = set(atom_types)
    pp_section= generate_pp_section(elements, pp)
    ion_file = cell_section + position_section + pp_section
    return(ion_file)

def write_file(fname, fcontent):
    with open(fname, 'w') as f:
        f.writelines(fcontent)

In [11]:
cell_params = np.array([[3.97, 0, 0], [0, 3.97, 0], [0, 0, 3.97]])
# write_cell_section(cell_param)

In [12]:
pos = np.array([[0.99670699, 0.95965284, 0.93381647],
       [3.09466771, 3.01576909, 1.03638038],
       [0.99267473, 3.08283329, 3.33323517],
       [2.85587579, 0.88164602, 2.63676545]])
atoms = ['Al', 'Al', 'Al', 'Al']
# write_position_section(atom_types, positions, 'CART')

In [13]:
# write_pp_section(set(atom_types), ['al_HC.lda.recpot'])

In [14]:
ion_file = generate_ion_file(cell_params, atoms, pos, 'CART', ['al_HC.lda.recpot'])
write_file('/home/misa/git_repositories/PROFESS/test/MD_NVE/test.ion', ion_file)

In [15]:
ion_file

['%BLOCK LATTICE_CART\n',
 '\t3.97\t0.0\t0.0\n',
 '\t0.0\t3.97\t0.0\n',
 '\t0.0\t0.0\t3.97\n',
 '%END BLOCK LATTICE_CART\n',
 '%BLOCK POSITIONS_CART\n',
 '\tAl\t0.99670699\t0.95965284\t0.93381647\n',
 '\tAl\t3.09466771\t3.01576909\t1.03638038\n',
 '\tAl\t0.99267473\t3.08283329\t3.33323517\n',
 '\tAl\t2.85587579\t0.88164602\t2.63676545\n',
 '%END BLOCK POSITIONS_CART\n',
 '%BLOCK SPECIES_POT\n',
 '\tAl\tal_HC.lda.recpot\n',
 '%END BLOCK SPECIES_POT\n']

#### write .ion file

In [3]:
import profess_io as pio
import numpy as np

In [9]:
cell_params = np.array([[3.97, 0, 0], [0, 3.97, 0], [0, 0, 3.97]])
pos = np.array([[0.99670699, 0.95965284, 0.93381647],
       [3.09466771, 3.01576909, 1.03638038],
       [0.99267473, 3.08283329, 3.33323517],
       [2.85587579, 0.88164602, 2.63676545]])
atoms = ['Al', 'Al', 'Al', 'Al']
file = pio.generate_ion_file(cell_params, atoms, pos, 'CART', ['al_HC.lda.recpot'])
pio.write_file('/home/misa/git_repositories/PROFESS/test/MD_NVE/test.ion', file)

#### read .out file

In [2]:
import profess_io as pio
pio.parse_out_file('/home/misa/git_repositories/PROFESS/test/ase_singlepoint/sp.out', 'TOTAL ENERGY')

-230.9273466291